In [ ]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))  # Should return your GPU model


True
Tesla T4


In [ ]:
!pip install -U bitsandbytes accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu

In [ ]:
#download the model and tokenizer
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig

MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
quant_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto",torch_dtype=torch.float16,quantization_config=quant_config)


print("Model and tokenizer downloaded successfully!")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Model and tokenizer downloaded successfully!


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 22.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.


In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM
from datasets import load_dataset
import torch
import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model
from transformers import BitsAndBytesConfig
import math

def compute_metrics(eval_pred):
    loss = eval_pred.loss
    perplexity = math.exp(loss)
    return {"perplexity": perplexity}

# Enable 8-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,  # Use 8-bit quantization
    llm_int8_enable_fp32_cpu_offload=True  # Offload CPU computations to save VRAM
)

# Load model and force it onto the current CUDA device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map={'': torch.cuda.current_device()},  # Assign model to GPU
    quantization_config=quantization_config  # Apply quantization
)
  # Move model to GPU




# Apply LoRA for efficient fine-tuning
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none"
)
model = get_peft_model(model, lora_config)

# Load tokenized dataset
from datasets import DatasetDict

# Load tokenized dataset
dataset = load_dataset("parquet", data_files={"data": "/content/combined-dataset.parquet"})

# Convert dataset to dictionary format
dataset = dataset["data"].train_test_split(test_size=0.1)

# Prepare data for conversational fine-tuning
dataset = DatasetDict({
    "train": dataset["train"].map(lambda x: {"labels": x["Response"] if "Response" in x else None}),
    "eval": dataset["test"].map(lambda x: {"labels": x["Response"] if "Response" in x else None})
})

# Ensure `labels` and `input_ids` are derived from the context and response
def preprocess_function(examples):
    # Tokenize the context and response separately
    model_inputs = tokenizer(examples["Context"], text_target=examples["Response"], padding="max_length", truncation=True)

    # Set labels to -100 where we don't need them to be predicted
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    # Create attention mask for the labels to ignore padding tokens
    labels_attention_mask = [
        [1 if label != tokenizer.pad_token_id else -100 for label in labels_row]
        for labels_row in model_inputs["labels"]
    ]
    model_inputs["labels_attention_mask"] = labels_attention_mask
    # # Shift labels for next token prediction (Causal LM)
    # model_inputs["labels"] = model_inputs["input_ids"][:, 1:].tolist()

    return model_inputs


# Apply preprocessing function
dataset = dataset.map(preprocess_function, batched=True)


# Define training arguments
training_args = TrainingArguments(
    output_dir="./fine_tuned_results_v2",
    per_device_train_batch_size=2,  # Lower batch size for 4GB GPU
    per_device_eval_batch_size=2,
    num_train_epochs=3,  # Train for 3 epochs
    save_total_limit=2,
    eval_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=100,
    save_strategy="epoch"
)


# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["eval"],

)


# Start training
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_model_v2")
print("✅ Fine-tuning complete! Model saved.")


Generating data split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/3160 [00:00<?, ? examples/s]

Map:   0%|          | 0/352 [00:00<?, ? examples/s]

Map:   0%|          | 0/3160 [00:00<?, ? examples/s]

Map:   0%|          | 0/352 [00:00<?, ? examples/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: xlogan-exe (xlogan-exe-presidency-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.081800,No log
2,0.070000,No log
3,0.077600,No log


✅ Fine-tuning complete! Model saved.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
from peft import PeftModel
import torch

# Define paths
BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Base model
FINETUNED_PATH = "./fine_tuned_model_v2"  # Where fine-tuned model will be saved

# Load base model
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="cpu")

# Load LoRA adapters and merge them
model = PeftModel.from_pretrained(model, FINETUNED_PATH)
model = model.merge_and_unload()  # Merge LoRA weights

# Save the full fine-tuned model
model.save_pretrained(FINETUNED_PATH)

# Save tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.save_pretrained(FINETUNED_PATH)

# Save configuration
config = AutoConfig.from_pretrained(BASE_MODEL)
config.save_pretrained(FINETUNED_PATH)

print("✅ Full fine-tuned model saved successfully!")


✅ Full fine-tuned model saved successfully!


In [ ]:
!zip -r fine_tuned_model_v2.zip ./fine_tuned_model_v2


  adding: fine_tuned_model_v2/ (stored 0%)
  adding: fine_tuned_model_v2/tokenizer.model (deflated 55%)
  adding: fine_tuned_model_v2/generation_config.json (deflated 29%)
  adding: fine_tuned_model_v2/README.md (deflated 66%)
  adding: fine_tuned_model_v2/model.safetensors (deflated 49%)
  adding: fine_tuned_model_v2/tokenizer_config.json (deflated 68%)
  adding: fine_tuned_model_v2/adapter_config.json (deflated 54%)
  adding: fine_tuned_model_v2/tokenizer.json (deflated 85%)
  adding: fine_tuned_model_v2/special_tokens_map.json (deflated 79%)
  adding: fine_tuned_model_v2/config.json (deflated 48%)
  adding: fine_tuned_model_v2/adapter_model.safetensors (deflated 8%)


In [ ]:
# prompt: generate a code to upload mu fine tune model to google drive by name fine_tuned_model2 and also include a progress bar to see the progress

from google.colab import drive
import shutil
import os
from tqdm import tqdm

# drive.mount('/content/drive')

# Define source and destination paths
source_dir = "./fine_tuned_model_v2"
destination_dir = "/content/drive/MyDrive/fine_tuned_model_v2"

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Get a list of files and directories to copy
items_to_copy = [f for f in os.listdir(source_dir)]

# Iterate over the files and directories using tqdm for the progress bar
with tqdm(total=len(items_to_copy), desc="Uploading to Google Drive") as pbar:
    for item in items_to_copy:
        source_path = os.path.join(source_dir, item)
        destination_path = os.path.join(destination_dir, item)

        if os.path.isdir(source_path):
          # Copy directories recursively
          shutil.copytree(source_path, destination_path, dirs_exist_ok=True)
        else:
          # Copy files
          shutil.copy2(source_path, destination_path)

        pbar.update(1)

print(f"✅ Model uploaded to {destination_dir}")


In [ ]:
# prompt: unmount my drive

drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')


Drive not mounted, so nothing to flush and unmount.
All changes made in this colab session should now be visible in Drive.
